<a href="https://colab.research.google.com/github/pramodith/llm_exploration/blob/colab/dynamic_prompt_token_dropping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install transformers
!pip install langchain
!pip install datasets
!pip install huggingface_hub

In [52]:
!pip install --upgrade transformers

     ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/7.9 MB 2.2 MB/s eta 0:00:04
      --------------------------------------- 0.2/7.9 MB 2.6 MB/s eta 0:00:03
     -- ------------------------------------- 0.4/7.9 MB 3.6 MB/s eta 0:00:03
     --- ------------------------------------ 0.6/7.9 MB 3.8 MB/s eta 0:00:02
     ---- ----------------------------------- 0.8/7.9 MB 4.2 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/7.9 MB 4.7 MB/s eta 0:00:02
     ------- -------------------------------- 1.4/7.9 MB 4.9 MB/s eta 0:00:02
     -------- ------------------------------- 1.7/7.9 MB 5.1 MB/s eta 0:00:02
     ---------- ----------------------------- 2.0/7.9 MB 5.5 MB/s eta 0:00:02
     ----------- ---------------------------- 2.3/7.9 MB 5.7 MB/s eta 0:00:01
     ------------- -------------------------- 2.7/7.9 MB 6.0 MB/s eta 0:00:01
     --------------- ------------------------ 3.1/7.9 MB 6.1 MB/s eta 0

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\pramo\\anaconda3\\envs\\PromptRiddler\\Lib\\site-packages\\~okenizers\\tokenizers.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [53]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List
import torch
from pprint import pprint
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datasets import load_dataset



ImportError: cannot import name 'get_session' from 'huggingface_hub.utils' (c:\Users\pramo\anaconda3\envs\PromptRiddler\lib\site-packages\huggingface_hub\utils\__init__.py)

In [3]:
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [4]:
def to_tokens_and_probs(model: AutoModelForCausalLM, tokenizer: AutoTokenizer, input_texts: List[str]):
    """
    This function takes a list of input texts and returns a list of tuples (token, prob) for each token in the input text.
    Reference: https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075/17

    Args:
        model (AutoModelForCausalLM): _description_
        tokenizer (AutoTokenizer): _description_
        input_texts (List[str]): _description_

    Returns:
        _type_: _description_
    """
    input_ids = tokenizer(input_texts, padding=True, return_tensors="pt").input_ids
    outputs = model(input_ids)
    probs = torch.softmax(outputs.logits, dim=-1).detach()

    # collect the probability of the generated token -- probability at index 0 corresponds to the token at index 1
    probs = probs[:, :-1, :]
    input_ids = input_ids[:, 1:]
    gen_probs = torch.gather(probs, 2, input_ids[:, :, None]).squeeze(-1)

    batch = []
    for input_sentence, input_probs in zip(input_ids, gen_probs):
        text_sequence = []
        for token, p in zip(input_sentence, input_probs):
            if token not in tokenizer.all_special_ids:
                text_sequence.append((tokenizer.decode(token), p.item()))
        batch.append(text_sequence)
    return batch

In [6]:
sample_prompt = ["The capital of France is Paris."]
token_probs = to_tokens_and_probs(model, tokenizer, sample_prompt)


In [7]:
token_probs

[[('The', 0.08746518939733505),
  (' capital', 8.743479702388868e-05),
  (' of', 0.14376786351203918),
  (' France', 0.005475870333611965),
  (' is', 0.24484333395957947),
  (' Paris', 0.006779632996767759),
  ('.', 0.3360886871814728)]]

Set your key to openai's API's using colab's secrets features.

In [9]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [32]:
chat = ChatOpenAI(temperature=0)

In [33]:
from pydantic import BaseModel
class ReadingComprehensionPrompt(BaseModel):
    system_message: str = SystemMessagePromptTemplate.from_template(f"You are a very smart student in a reading comprehension class." 
        "Your teacher is giving you a reading comprehension test. You are given a passage and a question."
        "You must answer the question based on the passage.")
    human_message: str = HumanMessagePromptTemplate.from_template("Passage: {passage}\nQuestion: {question}\nAnswer: ")

rc_prompt = ReadingComprehensionPrompt()

In [34]:
def get_llm_response(prompt: BaseModel, chat: ChatOpenAI, passage: str, question: str):
    chat_message = ChatPromptTemplate.from_messages([prompt.system_message, prompt.human_message])
    completed_prompt = chat_message.format_prompt(passage=passage, question=question).to_messages()
    return chat(completed_prompt).content

In [36]:
get_llm_response(rc_prompt, chat, "The capital of France is Paris.", "What is the capital of Australia?")

'The passage does not provide any information about the capital of Australia.'

In [43]:
def sample_squad_dataset(num_samples: int = 100):
    dataset = load_dataset("squad", split="test")
    return dataset.shuffle().select(range(num_samples))

In [44]:
squad_dataset = sample_squad_dataset()

NameError: name 'load_dataset' is not defined